# Heat Equation Porting Exercise

The following exercise will walk through the porting of a 3D heat equation code from Fortran to GT4Py.  The cell below contains a simple 3D heat equation code written in Fortran.  For this part of the exercise, the developer will do the following.

* Initialize Serialbox in the Fortran code
* Serialize the initial heat data `curr_Heat` and relevant variables to the Python/GT4Py port.
* Serialize the output heat data to compare to the Python/GT4Py port

After inserting the Serialbox calls, the developer can run the cell to create a source file called `heat.F90`.

In [52]:
%%writefile heat.F90

program heat

    implicit none

    integer :: ii, jj, kk, N, t_steps, curr_step, nx, ny, nz, nhalo

    real :: scale

    double precision, dimension(:,:,:), allocatable :: curr_Heat, future_Heat

    write(*,*) "Enter the Number of discretization cells N: "
    read(*,*) N
    write(*,*) "Enter number of time steps : "
    read(*,*) t_steps

    write(*,*) "N = ", N
    write(*,*) "Time steps = ", t_steps

    ! Note : Currently, the default domain is a cube, so the number of 
    !        discretization cells in x, y, and z are the same
    nhalo = 1
    nx = N
    ny = N
    nz = N
    allocate(curr_Heat(nx+2*nhalo,ny+2*nhalo,nz), future_Heat(nx+2*nhalo,ny+2*nhalo,nz))
   
    !***Insert Serialbox calls here for initialization***
    !$ser init directory='./data' prefix="HEAT" unique_id=.true.
    !$ser mode write
    !$ser on

    curr_Heat   = 0.0
    future_Heat = 0.0

    scale = 0.1

    curr_Heat(nhalo+1:nx-nhalo, nhalo+1:ny-nhalo, nhalo+1:nz-nhalo) = 1.0

    !***Insert Serialbox calls to create a savepoint to save initial starting data***
    !$ser savepoint 'starting_heat'
    !$ser data init_T=curr_Heat scale=scale t_steps=t_steps nhalo=nhalo
    do curr_step = 1,t_steps
        do kk = 2,nz-1
            do jj = nhalo,ny-nhalo-1
                do ii = nhalo+1,nx-nhalo-1
                    future_Heat(ii,jj,kk) = curr_Heat(ii,jj,kk)              &
                                        + scale * (curr_Heat(ii-1,jj,kk)   &
                                                    -2.0*curr_Heat(ii,jj,kk) &
                                                    +curr_Heat(ii+1,jj,kk))  &
                                        + scale * (curr_Heat(ii,jj-1,kk)   &
                                                    -2.0*curr_Heat(ii,jj,kk) &
                                                    +curr_Heat(ii,jj+1,kk))  &
                                        + scale * (curr_Heat(ii,jj,kk-1)   &
                                                    -2.0*curr_Heat(ii,jj,kk) &
                                                    +curr_Heat(ii,jj,kk+1))
                enddo
            enddo
        enddo

        curr_Heat = future_Heat

    enddo
    
    !***Insert Serialbox calls to create a savepoint to write finalized data***
    !$ser savepoint 'final_heat'
    !$ser data final_T=curr_Heat
    !$ser cleanup

end program

Overwriting heat.F90


After adding the Serialbox calls, the developer can run the cell below to perform the following:

* Call `pp_ser.py` to substitute the Serialbox "directives" with Serialbox library calls in the Fortran code and output the result into a new Fortran source file called `s_heat.F90`
* Compile `s_heat.F90` and link with the Serialbox libraries into a binary called `HEAT`
* Run the `HEAT` binary 

Note that the code implementation has two inputs (size of the domain in cells and number of time steps) that must be listed on individual lines in the cell below when running the `HEAT` binary.  The developer can try different entries to help with debugging if necessary.

In [53]:
%%bash

[ -f HEAT ] && rm HEAT
[ -f s_heat.F90 ] && rm s_heat.F90

python3 ${SERIALBOX_ROOT}/python/pp_ser/pp_ser.py -s -v --output=s_heat.F90 heat.F90

gfortran -O3 -cpp -DSERIALIZE -o HEAT s_heat.F90 \
   -I${SERIALBOX_ROOT}/include \
    ${SERIALBOX_ROOT}/lib/libSerialboxFortran.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxC.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxCore.a \
 -lpthread -lstdc++ -lstdc++fs
./HEAT
40
10


Processing file heat.F90
 Enter the Number of discretization cells N: 
 Enter number of time steps : 
 N =           40
 Time steps =           10
 >>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<
 >>> WARNING: SERIALIZATION IS ON <<<
 >>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<



After ensuring that you are able to run the code and successfully serialize the data, you can start to write your Python/GT4Py port of the heat equation code in the cell below.  To compare solutions between the Python and the Fortran, use the `np.allclose()` function.

Here are a couple of things to keep in mind.

* Python array indices start at 0
* While it's not explicitly stated, the heat equation example has an implied boundary condition where the "surface" temperatures are set to 0

## Python/GT4py solution

In [58]:
import os
import sys
sys.path.append(os.environ.get('SERIALBOX_ROOT') + '/python')
import serialbox as ser
import numpy as np
import gt4py
import gt4py.gtscript as gtscript
import gt4py.storage as gt_storage

backend = "numpy"
F_TYPE = np.float64

@gtscript.stencil(backend=backend)
def heat_update(curr_T   : gtscript.Field[F_TYPE],
                future_T : gtscript.Field[F_TYPE],
                *,
                scale    : np.float32):
    with computation(PARALLEL), interval(...):
        future_T = curr_T + scale * (curr_T[-1,0,0] - 2.0*curr_T + curr_T[1,0,0]) \
                          + scale * (curr_T[0,-1,0] - 2.0*curr_T + curr_T[0,1,0]) \
                          + scale * (curr_T[0,0,-1] - 2.0*curr_T + curr_T[0,0,1])

        curr_T = future_T

serializer = ser.Serializer(ser.OpenModeKind.Read, './data', 'HEAT')

sp = serializer.get_savepoint('starting_heat')

curr_T = serializer.read('init_T', sp[0])
scale     = serializer.read('scale', sp[0])[0]
t_steps   = serializer.read('t_steps', sp[0])[0]
nhalo = serializer.read('nhalo', sp[0])[0]
origin=(nhalo,nhalo,1)
full_shape = curr_T.shape
nx = full_shape[0] - 2 * nhalo
ny = full_shape[1] - 2 * nhalo
nz = full_shape[2] - 2
curr_T_gt = gt_storage.from_array(curr_T, 
                                  backend=backend, 
                                  default_origin=(nhalo,nhalo,1))

future_T_gt = gt_storage.zeros(backend=backend, 
                               dtype=F_TYPE, 
                               shape=full_shape,
                               default_origin=origin)

for curr_step in range(t_steps):
    print(curr_T_gt.shape, future_T_gt.shape, nx,ny,nz, origin, scale)
    heat_update(curr_T=curr_T_gt,
                future_T=future_T_gt,
                scale=scale,
                origin=origin,
                domain=(nx, ny, nz))

sp = serializer.get_savepoint('final_heat')
final_T = serializer.read('final_T', sp[0])

if np.array_equal(final_T,curr_T_gt):
    print("Solution is valid!")
else:
    for i in range(full_shape[0]):
        for j in range(full_shape[1]):
            for k in range(2):
                if final_T[i,j,k] != curr_T_gt[i, j, k]:
                    print(i,j , k, final_T[i,j,k], curr_T_gt[i, j, k],final_T[i,j,k] - curr_T_gt[i, j, k])
    raise Exception("Solution is NOT valid")

(42, 42, 40) (42, 42, 40) 40 40 38 (1, 1, 1) 0.1
(42, 42, 40) (42, 42, 40) 40 40 38 (1, 1, 1) 0.1
(42, 42, 40) (42, 42, 40) 40 40 38 (1, 1, 1) 0.1
(42, 42, 40) (42, 42, 40) 40 40 38 (1, 1, 1) 0.1
(42, 42, 40) (42, 42, 40) 40 40 38 (1, 1, 1) 0.1
(42, 42, 40) (42, 42, 40) 40 40 38 (1, 1, 1) 0.1
(42, 42, 40) (42, 42, 40) 40 40 38 (1, 1, 1) 0.1
(42, 42, 40) (42, 42, 40) 40 40 38 (1, 1, 1) 0.1
(42, 42, 40) (42, 42, 40) 40 40 38 (1, 1, 1) 0.1
(42, 42, 40) (42, 42, 40) 40 40 38 (1, 1, 1) 0.1
1 0 1 0.08164675798230031 0.0 0.08164675798230031
1 1 1 0.1621829996992753 0.12704776473675244 0.035135234962522854
1 2 1 0.21923255085651064 0.20869452271905273 0.010538028137457911
1 3 1 0.24609355054695786 0.24382975768157553 0.002263792865382336
1 4 1 0.2547208471352497 0.2543677858190334 0.00035306131621626324
1 5 1 0.2566715371869713 0.2566315786844157 3.995850255555933e-05
1 6 1 0.25698786600089946 0.2569846400006319 3.2260002675310595e-06
1 7 1 0.2570247765032057 0.25702459850318743 1.780000182804

33 31 1 0.5109362796261824 0.5112657966442894 -0.0003295170181070173
33 32 1 0.5109056175239192 0.5112613776439004 -0.00035576011998128543
33 33 1 0.5105475867039652 0.5112035031398952 -0.0006559164359299929
33 34 1 0.5076712387934599 0.5106726398128038 -0.0030014010193438745
33 35 1 0.4915720798375196 0.5072057096963338 -0.015633629858814224
33 36 1 0.42933495538261923 0.49113731542042643 -0.0618023600378072
33 37 1 0.2678358727857267 0.43942166679763733 -0.17158579401191065
33 38 1 0.0 0.32876505634274394 -0.32876505634274394
33 39 1 0.0 0.18184505696564057 -0.18184505696564057
33 40 1 0.0 0.06777939789828322 -0.06777939789828322
34 0 1 0.16132139942994753 0.0 0.16132139942994753
34 1 1 0.3226173770194129 0.25667171518698956 0.06594566183242334
34 2 1 0.43594459885728315 0.4188206167512603 0.01712398210602284
34 3 1 0.48792978819224186 0.48655444464617015 0.0013753435460717167
34 4 1 0.5040034323681161 0.5060821440378152 -0.002078711669699085
34 5 1 0.5074684282843476 0.5100792751812

Exception: Solution is NOT valid

## Python-only solution

In [55]:
import os
import sys
sys.path.append(os.environ.get('SERIALBOX_ROOT') + '/python')
import serialbox as ser
import numpy as np

serializer = ser.Serializer(ser.OpenModeKind.Read, './data', 'HEAT')
print('here')
sp = serializer.get_savepoint('starting_heat')

curr_T = serializer.read('init_T', sp[0])
scale     = serializer.read('scale', sp[0])
t_steps   = serializer.read('t_steps', sp[0])

future_T = np.zeros((curr_T.shape[0],
                        curr_T.shape[1],
                        curr_T.shape[2]),dtype='float32')

for curr_step in range(t_steps[0]):
    for ii in range(1,curr_T.shape[2]-1):
        for jj in range(1, curr_T.shape[1]-1):
            for kk in range(1, curr_T.shape[2]-1):
                future_T[ii,jj,kk] = curr_T[ii,jj,kk] \
                  + scale[0] * (curr_T[ii-1,jj,   kk]   - 2.0*curr_T[ii,jj,kk] + curr_T[ii+1,jj,  kk]) \
                  + scale[0] * (curr_T[ii,  jj-1, kk]   - 2.0*curr_T[ii,jj,kk] + curr_T[ii,  jj+1,kk]) \
                  + scale[0] * (curr_T[ii,  jj,   kk-1] - 2.0*curr_T[ii,jj,kk] + curr_T[ii,  jj,  kk+1])


    curr_T[:,:,:] = future_T[:,:,:]
print('final')
sp = serializer.get_savepoint('final_heat')
final_T = serializer.read('final_T', sp[0])

if np.allclose(final_T,curr_T):
    print("Solution is valid!")
else:
    raise Exception("Solution is NOT valid")

here
final


Exception: Solution is NOT valid